#RETO EQUIPO 6

* Melissa Aurora Fadanelli Ordaz
* Nadia Paola Ferro Gallegos
* Jorge Rojas Rivas

#Importación de librerías

In [ ]:
!pip install nltk scikit-learn

In [ ]:
import os
import pandas as pd
import nltk
import re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise
from nltk.corpus import stopwords
import numpy as np
from tabulate import tabulate

#Preparación de fuente de datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Extracción de archivo zip
!unzip "/content/drive/MyDrive/RetoDesarrollo/dataset.zip" -d "/content/drive/MyDrive/RetoDesarrollo/"

Archive:  /content/drive/MyDrive/RetoDesarrollo/dataset.zip
   creating: /content/drive/MyDrive/RetoDesarrollo/dataset/test/
  inflating: /content/drive/MyDrive/RetoDesarrollo/dataset/test/FID-001.txt  
  inflating: /content/drive/MyDrive/RetoDesarrollo/dataset/test/FID-002.txt  
  inflating: /content/drive/MyDrive/RetoDesarrollo/dataset/test/FID-003.txt  
  inflating: /content/drive/MyDrive/RetoDesarrollo/dataset/test/FID-004.txt  
  inflating: /content/drive/MyDrive/RetoDesarrollo/dataset/test/FID-005.txt  
  inflating: /content/drive/MyDrive/RetoDesarrollo/dataset/test/FID-006.txt  
  inflating: /content/drive/MyDrive/RetoDesarrollo/dataset/test/FID-007.txt  
  inflating: /content/drive/MyDrive/RetoDesarrollo/dataset/test/FID-008.txt  
  inflating: /content/drive/MyDrive/RetoDesarrollo/dataset/test/FID-009.txt  
  inflating: /content/drive/MyDrive/RetoDesarrollo/dataset/test/FID-010.txt  
  inflating: /content/drive/MyDrive/RetoDesarrollo/dataset/test/FID-011.txt  
  inflating: /con

In [ ]:
datas = os.listdir('/content/drive/MyDrive/RetoDesarrollo/dataset/train')
datas.sort()
#print(datas)

#Lectura de textos de referencia

In [ ]:
#Función para leer todos los textos y procesarlos
def read_dic(data):
  files = []
  df = pd.read_csv("/content/drive/MyDrive/RetoDesarrollo/file_lemmat.csv")
  cont = 0

  for text in data:
    if cont >= len(df):
      break

    df.at[cont, 'Name'] = text

    hasVec=verify_lemmat(cont, text)

    if hasVec is None:
      #Se realiza todo el proceso
      file_path = '/content/drive/MyDrive/RetoDesarrollo/dataset/train/' + text
      with open(file_path, 'r') as file:
        rFile = file.readlines()
        #Proceso para limpieza
        cFile=clean_data(rFile, file_path)

        #Aplicar Lematización
        lemmed = lem_file(cFile)
        #print(lemmed)
        #Vectorizar
        #vecs = vectorize_file(lemmed)
        #print(vecs)
        df.at[cont, 'Lemmatization'] = lemmed

        files.append(lemmed)

    cont += 1
  df.to_csv("/content/drive/MyDrive/RetoDesarrollo/file_lemmat.csv", index=False)

  #print(files)
  return files


In [ ]:
#Función para verificar si el vector se encuentra en el csv
def verify_lemmat(cont, name):
  df = pd.read_csv("/content/drive/MyDrive/RetoDesarrollo/file_lemmat.csv") #Leer el csv

  if pd.isna(df.at[cont, 'Lemmatization']):
    return None
  else:
    return df.at[cont, 'Lemmatization']


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
#Funciones de limpieza de datos
def clean_data(file, file_path):
  #print(file)
  text = ''.join(file)
  #Eliminación de signos de puntuación
  clean_text = re.sub(r'[^\w\s]', '', text)
  stop_words = set(stopwords.words('english'))

  #print(clean_text)

  #Tokenización
  nltk_tokens = nltk.word_tokenize(clean_text)

   # Eliminar las stopwords de los tokens
  filtered_tokens = [word for word in nltk_tokens if word.lower() not in stop_words]

  #print(f"Filtered Tokens for {file_path}: {filtered_tokens}")
  return filtered_tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def lem_file(tokens):
    lemmed_file = [lemmatizer.lemmatize(token) for token in tokens]
    #print(lemmed_file)
    vector = np.array(lemmed_file)
    vector_str = ','.join(map(str, vector))
    #print(vector_str)
    return vector_str


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
files = read_dic(datas)

#Comparación de texto nuevo

In [ ]:
#Función para obtener el vector del nuevo archivo
def get_lemmed(file_path):
  with open(file_path, 'r') as newFile:
    rNewFile = newFile.readlines()
    #print(rNewFile)
    #Proceso para limpieza
    cNewFile=clean_data(rNewFile, file_path)

    #Aplicar Lematización
    lemmedNew = lem_file(cNewFile)
    #print(lemmedNew)
    return lemmedNew

In [ ]:
#Función para vectorizar
def vectorize_file(textsList):
  vectorizer = CountVectorizer()
  vector = vectorizer.fit_transform(textsList).toarray()
  #print(vector)
  return vector

In [ ]:
def getFullText(tokenList):
  newSent = ''
  for word in tokenList:
    #print(word)
    newSent += word + " "
    #print(newSent)
  #print(newSent)
  return newSent

In [ ]:
# Función para calcular las tres similitudes de coseno más altas
def top_3_similarity(simil):
    top_3_similarities = sorted(simil, key=lambda x: x[0], reverse=True)[:3]

    #print("Top 3 Similitudes de coseno con los textos:")
    for sim, name in top_3_similarities:
      similarity_percent = sim * 100
      #print(f"Similitud: {sim}, Porcentaje: {similarity_percent:.2f}%, Texto: {name}")

    return top_3_similarities

In [ ]:
np.set_printoptions(threshold=np.inf)
def measure_distance(newLemmed):
  simil = []
  df = pd.read_csv("/content/drive/MyDrive/RetoDesarrollo/file_lemmat.csv") #Leer el csv
  fullNew = getFullText(newLemmed.split(','))
  for i in range(0, len(df)):
    bothTexts = []
    text_name = df.at[i, 'Name']
    oldLemm = df.at[i, 'Lemmatization']
    #print(oldLemm.split(','))
    fullOld = getFullText(oldLemm.split(','))
    bothTexts.append(fullOld)
    bothTexts.append(fullNew)
    #print(bothTexts)
    vec = vectorize_file(bothTexts)
    #print(vec)

    cosSim = pairwise.cosine_similarity(vec)[0, 1]  # Obtener la similitud de coseno
    simil.append((cosSim, text_name))
    #print(cosSim)
  #print(simil)
  return simil

In [91]:
def final_veredict(umbral, tupleList):
  # Filtrar las similitudes para seleccionar solo aquellas por encima del umbral
  plagiarism_cases = [(sim, name) for sim, name in tupleList if sim > umbral]
  return plagiarism_cases


In [94]:
#Ingresar nombre del archivo que se quiere utilizar
#new_file = input("Ingrese el nombre del texto: ")
new_file = "FID-005"
file_path = '/content/drive/MyDrive/RetoDesarrollo/dataset/test/' + new_file + '.txt'

newLemmed = get_lemmed(file_path)
similarities = measure_distance(newLemmed)
top_3_similarities = top_3_similarity(similarities)
final_veredict(0.4, top_3_similarities)

[(0.7911865309325848, 'org-023.txt'), (0.4039215399576591, 'org-059.txt')]

In [95]:
pruebas = os.listdir('/content/drive/MyDrive/RetoDesarrollo/dataset/test')
pruebas.sort()

In [105]:
def print_results_as_table(results):
    headers = ["Archivo", "Plagiado", "Similitud", "Texto"]
    table_data = []

    for text, veredict in results:
      if veredict:
        plagio_status = "Sí"
        # Procesar el veredicto para obtener similitud y textos
        for similarity, similar_text in veredict:
          table_data.append([text, plagio_status, f"{similarity:.2%}", similar_text])
      else:
        table_data.append([text, "No", "", ""])

    print(tabulate(table_data, headers=headers, tablefmt="grid"))

In [107]:
def read_pruebas(data):
  files = []
  cont = 0
  pruebas

  for text in data:
    if cont >= len(pruebas):
      break

    file_path = '/content/drive/MyDrive/RetoDesarrollo/dataset/test/' + text
    newLemmed = get_lemmed(file_path)
    similarities = measure_distance(newLemmed)
    #print(text)
    top_3_similarities = top_3_similarity(similarities)
    #final_veredict(0.3, top_3_similarities)
    veredict = final_veredict(0.4, top_3_similarities)
    #final_veredict(0.5, top_3_similarities)
    #final_veredict(0.6, top_3_similarities)
    #final_veredict(0.7, top_3_similarities)
    #print('\n')
    files.append((text, veredict))

    cont += 1

  print_results_as_table(files)
  #print(files)
  #return files


In [109]:
read_pruebas(pruebas)

+-------------+------------+-------------+-------------+
| Archivo     | Plagiado   | Similitud   | Texto       |
+=============+============+=============+=============+
| FID-001.txt | No         |             |             |
+-------------+------------+-------------+-------------+
| FID-002.txt | No         |             |             |
+-------------+------------+-------------+-------------+
| FID-003.txt | No         |             |             |
+-------------+------------+-------------+-------------+
| FID-004.txt | No         |             |             |
+-------------+------------+-------------+-------------+
| FID-005.txt | Sí         | 79.12%      | org-023.txt |
+-------------+------------+-------------+-------------+
| FID-005.txt | Sí         | 40.39%      | org-059.txt |
+-------------+------------+-------------+-------------+
| FID-006.txt | No         |             |             |
+-------------+------------+-------------+-------------+
| FID-007.txt | No         |   

A pesar de que se eligen los 3 archivos con la mayor similitud, no significa que los 3 presenten plagio, para esto se tiene que analizar el porcentaje de similitud, que a partir de un .4 o .5 podrían representar plagio, menor a eso se podría considerar coincidencias.

Para corroborar esto se realizó un método de prueba y error, tomando las mediciones que entren dentro de un umbral de 30%, 40%, 50%, 60% y 70%. Observando los resultados y comparando con la información que tenemos determinamos que el umbral más adecuado sería el de 40%.